In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

TensorFlow 2.x selected.


In [2]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [3]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

193 train examples
49 validation examples
61 test examples


In [0]:
def ds_to_dataframe(dataframe,shuffle=True,batch_size=35):
  dataframe=dataframe.copy()
  labels=dataframe.pop('target')
  ds=tf.data.Dataset.from_tensor_slices((dict(dataframe),labels))
  if shuffle:
    ds=ds.shuffle(buffer_size=len(dataframe))
  ds=ds.batch(batch_size)
  return ds

In [0]:
train_ds=ds_to_dataframe(train,batch_size=5)
test_ds=ds_to_dataframe(test,batch_size=5,shuffle=False)
val_ds=ds_to_dataframe(val,batch_size=5,shuffle=False)

In [0]:
example_batch = next(iter(train_ds))[0]

In [0]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [11]:
age = feature_column.numeric_column("age")
demo(age)

[[45.]
 [63.]
 [40.]
 [54.]
 [63.]]


In [0]:
feature_columns=[]

In [0]:
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
  feature_columns.append(feature_column.numeric_column(header))

In [14]:
# indicator cols
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='trestbps', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='chol', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='thalach', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='oldpeak', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='slope', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ca', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [0]:
thal=feature_column.categorical_column_with_vocabulary_list('thal', ['fixed', 'normal', 'reversible'])
thal_one_hot=feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [17]:
feature_layer

In [18]:
model=tf.keras.Sequential([
    feature_layer,
    layers.Dense(128,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')
    
    
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'],
              run_eagerly=True)

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Epoch 1/5


W0822 20:12:51.538667 140711618045824 deprecation.py:323] From /tensorflow-2.0.0b1/python3.6/tensorflow/python/feature_column/feature_column_v2.py:2655: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0822 20:12:51.573034 140711618045824 deprecation.py:323] From /tensorflow-2.0.0b1/python3.6/tensorflow/python/feature_column/feature_column_v2.py:4215: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0822 20:12:51.574692 140711618045824 deprecation.py:323] From /tensorflow-2.0.0b1/python3.6/tensorflow/python/feature_column/feature_column_v2.py:4270: VocabularyListCategoricalColumn._num_buckets (

39/39 [==============================] - 2s 43ms/step - loss: 1.7386 - accuracy: 0.6820 - val_loss: 0.6226 - val_accuracy: 0.7551
Epoch 2/5
39/39 [==============================] - 1s 20ms/step - loss: 0.9084 - accuracy: 0.7133 - val_loss: 0.8265 - val_accuracy: 0.6531
Epoch 3/5
39/39 [==============================] - 1s 21ms/step - loss: 0.8807 - accuracy: 0.6930 - val_loss: 0.6060 - val_accuracy: 0.6939
Epoch 4/5
39/39 [==============================] - 1s 21ms/step - loss: 0.7843 - accuracy: 0.7674 - val_loss: 0.7210 - val_accuracy: 0.6939
Epoch 5/5
39/39 [==============================] - 1s 21ms/step - loss: 0.7629 - accuracy: 0.7419 - val_loss: 0.5640 - val_accuracy: 0.7143


In [21]:
loss,accuracy=model.evaluate(test_ds)
print(accuracy)

13/13 [==============================] - 0s 9ms/step - loss: 0.5962 - accuracy: 0.7541
0.75409836
